In [ ]:
import time
from selenium import webdriver

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
import cv2
import imutils
import pickle
import numpy as np
num_classes = 32
NNmodel = Sequential()
NNmodel.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
NNmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NNmodel.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
NNmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NNmodel.add(Flatten())
NNmodel.add(Dense(512, activation="relu"))
NNmodel.add(Dense(num_classes, activation="softmax"))
NNmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
NNmodel.load_weights("weights.h5")
label_binarizer = pickle.load(open("binarizer.pkl",'rb'))

def resize_image_to_dimensions(image, desired_width, desired_height):
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=desired_width)
    else:
        image = imutils.resize(image, height=desired_height)
    padWidth = int((desired_width - image.shape[1]) / 2.0)
    padHeight = int((desired_height - image.shape[0]) / 2.0)
    imageWBorder = cv2.copyMakeBorder(image, padHeight, padHeight, padWidth, padWidth,
        cv2.BORDER_REPLICATE)
    imageWBorderResized = cv2.resize(imageWBorder, (desired_width, desired_height))
    return imageWBorderResized

def findBoundingRectanglesOfContours(contours):
    letter_bounding_rectangles = []
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append((x, y, w, h))
    return letter_bounding_rectangles

def preprocessCAPTCHA(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grayWBorder = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    preprocessed = cv2.threshold(grayWBorder, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return grayWBorder, preprocessed

def CAPTCHAtoGrayscaleAndBoundingRectangles(captcha_image_file):
    image = cv2.imread(captcha_image_file)
    gray, preprocessed = preprocessCAPTCHA(image)
    contours = cv2.findContours(preprocessed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0]
    letter_bounding_rectangles = findBoundingRectanglesOfContours(contours)
    letter_bounding_rectangles = sorted(letter_bounding_rectangles, key=lambda x: x[0])
    return gray, letter_bounding_rectangles

def CAPTCHAImageToPrediction(CAPTCHA):
    gray, letter_bounding_rectangles = CAPTCHAtoGrayscaleAndBoundingRectangles(CAPTCHA)
    predictions = []

    for letter_bounding_rectangle in letter_bounding_rectangles:
        x, y, w, h = letter_bounding_rectangle
        letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
        letter_image = resize_image_to_dimensions(letter_image, 20, 20)
        letter_image = np.expand_dims(letter_image, axis=2)
        letter_image = np.expand_dims(letter_image, axis=0)
        prediction = NNmodel.predict(letter_image)
        letter = label_binarizer.inverse_transform(prediction)[0]
        predictions.append(letter)

    predicted_captcha_text = "".join(predictions)
    return predicted_captcha_text

In [3]:
import time
from selenium import webdriver
import urllib.request

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options) 
driver.get('https://ml4cs.com/CAPTCHA/index.php/break-me-use-a-bot-to-crack-the-captcha/');
window_handle = driver.window_handles[-1]
time.sleep(2)
driver.switch_to.default_content()
captcha_image = driver.find_element_by_css_selector(".wpcf7-captcha-captcha-170")
src = captcha_image.get_attribute("src");
urllib.request.urlretrieve(src, "captcha.png")
pred = CAPTCHAImageToPrediction("captcha.png")
captcha_input = driver.find_element_by_name("captcha-170")
captcha_input.send_keys(pred)
captcha_input.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()